In [1]:
import sionna.rt
from sionna.rt import load_scene, PlanarArray, Receiver, Transmitter

import mitsuba as mi
import drjit as dr
import numpy as np
import torch

In [2]:
# scene = load_scene(filename="cubes/cubes.xml")
scene = load_scene(filename="cubes/cubes.xml")
scene.rx_array = PlanarArray(num_rows=1,
                          num_cols=1,
                          vertical_spacing=0.5,
                          horizontal_spacing=0.5,
                          polarization="V",
                          pattern="iso")

scene.tx_array = PlanarArray(num_rows=1,
                          num_cols=1,
                          vertical_spacing=0.5,
                          horizontal_spacing=0.5,
                          polarization="V",
                          pattern="iso")


In [3]:
# scene.preview()

In [4]:
mi_scene = scene.mi_scene

In [5]:
shapes = mi_scene.shapes_dr()

# Will throw an error if there is a non-mesh shape
shapes = mi.MeshPtr(shapes)

face_normals = [
    s.face_normal(dr.arange(dr.cuda.UInt, s.face_count())).torch().T for s in shapes
]
face_normals = torch.vstack(face_normals)

# Find the average between the 3 vertices of a triangle mesh
# This will be the source for ray intersection testing
face_averages = []
for s in shapes:
    indices = dr.arange(dr.cuda.UInt, s.face_count())
    vertex_indices = dr.ravel(s.face_indices(indices), order='F')
    vertices = s.vertex_position(vertex_indices)
    vertices = vertices.torch().T.reshape(-1, 3, 3)
    face_averages.append(vertices)

face_averages = torch.vstack(face_averages)
face_averages = face_averages.sum(1) / 3

# print(face_averages.shape)
# print(face_averages)

In [6]:
# i = 0
# for face_avg in face_averages[3:6]:
#     # print(face_avg)
#     rx = Receiver(name=f"rx{i}", position=face_avg)
#     scene.add(rx)
#     i += 1

# # rx = Receiver(position=face_averages)
# # scene.add(rx)

In [7]:
# offset_normals = face_normals + face_averages

# i = 0
# for face_n in offset_normals:
#     # print(face_n)
#     tx = Transmitter(name=f"tx{i}", position=face_n)
#     scene.add(tx)
#     i += 1
# # tx = Transmitter(position=offset_normals)
# # scene.add(tx)

# scene.preview()

In [8]:
face_averages = mi.Point3f(face_averages.T)
face_normals = mi.Vector3f(face_normals.T)
print(face_averages)
print(face_normals)


[[-0.333333, 4.05719, 0.471405],
 [0.333333, 4.5286, 0.942809],
 [1, 5, 0.471405],
 .. 30 skipped ..,
 [1, -0.333333, -0.333333],
 [-0.333333, 1, 0.333333],
 [0.333333, 1, -0.333333]]
[[3.43526e-10, -0.707107, 0.707107],
 [0, -0.707107, 0.707107],
 [1, 0, 0],
 .. 30 skipped ..,
 [1, 0, 0],
 [0, 1, 0],
 [0, 1, 0]]


In [9]:
# Generate rays to be shot from each surface in 180 cone
# Currently generates 360 then filters based on dot product

from sionna.rt.utils import fibonacci_lattice, spawn_ray_from_sources
ray_per_source = 10

sources = face_averages + (face_normals * 0.0001)
rays = spawn_ray_from_sources(fibonacci_lattice, ray_per_source, sources) 
print(rays)

norms_expanded = dr.repeat(face_normals, ray_per_source)
print(norms_expanded)
# shouldn't this be positive dot prod???
active = dr.dot(rays.d, norms_expanded) > 0
print(active)

Ray[
  o=[[-0.333333, 4.05712, 0.471475],
     [-0.333333, 4.05712, 0.471475],
     [-0.333333, 4.05712, 0.471475],
     [-0.333333, 4.05712, 0.471475],
     [-0.333333, 4.05712, 0.471475],
        350 skipped ..,
     [0.333333, 1.0001, -0.333333],
     [0.333333, 1.0001, -0.333333],
     [0.333333, 1.0001, -0.333333],
     [0.333333, 1.0001, -0.333333],
     [0.333333, 1.0001, -0.333333]],
  d=[[0, 0, 1],
     [-0.463465, -0.424572, 0.777778],
     [0.0726926, 0.828296, 0.555556],
     [0.573642, -0.748214, 0.333333],
     [-0.978616, 0.173103, 0.111111],
        350 skipped ..,
     [0.838531, 0.533405, -0.111111],
     [-0.244757, -0.910485, -0.333333],
     [-0.383235, 0.737895, -0.555556],
     [0.5904, -0.215613, -0.777778],
     [-0, -0, -1]],
  maxt=[3.40282e+38],
  time=[0, 0, 0, 0, 0,    350 skipped .., 0, 0, 0, 0, 0],
  wavelengths=[]
]
[[3.43526e-10, -0.707107, 0.707107],
 [3.43526e-10, -0.707107, 0.707107],
 [3.43526e-10, -0.707107, 0.707107],
 .. 354 skipped ..,
 [0, 1, 

In [10]:
interactions = mi_scene.ray_intersect_preliminary(rays, active=active)

count = 0
for s in mi_scene.shapes():
    count += s.primitive_count()

a=dr.select(interactions.is_valid() & (interactions.t > 1.5), interactions.prim_index, dr.cuda.UInt(count)).numpy()
b=dr.select(interactions.is_valid(), interactions.t, dr.cuda.Float32(-1)).numpy()

# for i in range(len(a)):
#     print(a[i], b[i])



# for i, b in enumerate(a):
#     print(b, interactions.prim_index[i])


# Need to check to make sure ray is intersecting with the correct side of plane!!!!!!
# Should just be checking to make sure [ray direction] dot [norm is positive] > threshold? 

In [11]:

sets = [set() for _ in range(count)]

for i in range(len(a)):
    vis = a[i]
    if vis < count:
        sets[i // ray_per_source].add(vis)

sets
for i, s in enumerate(face_averages.numpy().T):
    print(s, sets[i])

scene

[-0.33333334  4.0571914   0.47140452] set()
[0.33333334 4.528596   0.94280905] {10}
[1.         5.         0.47140452] set()
[ 1.          5.         -0.47140452] set()
[ 0.33333334  4.0571914  -0.47140452] set()
[-0.33333334  4.528596   -0.94280905] set()
[-0.33333334  5.942809   -0.47140452] set()
[ 0.33333334  5.4714046  -0.94280905] set()
[-0.33333334  5.4714046   0.94280905] set()
[0.33333334 5.942809   0.47140452] set()
[-1.        4.528596  0.      ] set()
[-1.         5.4714046  0.       ] set()
[-4.8267746 -1.0799167  2.8387704] set()
[-4.083797  -0.5973266  3.1612298] set()
[-3.9402404   0.29087383  2.8794847 ] {5}
[-4.5396614  0.6964841  2.2752793] set()
[-4.9194956  -0.34518716  1.9528196 ] set()
[-5.7893386  -0.11522536  2.2345653 ] set()
[-5.6799254   0.75079745  3.4429755 ] set()
[-5.409504    0.92644584  2.5570247 ] set()
[-4.7173615  -0.21389394  4.0471807 ] set()
[-4.573805   0.6743064  3.765435 ] set()
[-5.696617   -0.84995496  3.1205158 ] set()
[-5.823482   -0.13740

interactions is 1d array num faces x rays per source long
reshape

In [12]:
sizes = [len(s) for s in sets]
face_averages_np = face_averages.numpy().T
starts = np.repeat(face_averages_np, repeats=sizes, axis=0)
print(starts.shape)

ends_indices = []
for s in sets:
    for shape in s:
        ends_indices.append(shape)

ends_indices = np.array(ends_indices)
ends = face_averages_np[ends_indices]
ends.shape

(2, 3)


(2, 3)

In [13]:
# This cell is lowkey pretty illegal but here's to being easier to implement
## THIS REQUIRES persist=True in call self._add_child(mesh, pmin, pmax, persist=True) in _plot_lines (preview.py)

# starts = np.array([[0,0,0],
#                    [1, 1, 1]])
# ends_indices = np.array([[5,5,5],
#                  [-100, -100, 0]])
scene.preview()

colors = np.array([1.0, 0.0, 0.0], dtype=np.float32)
colors = np.tile(colors, (starts.shape[0], 1))
# print(colors)
# print(colors.dtype)
width = np.array(1.0)
scene._preview_widget._plot_lines(starts, ends, colors, width)

In [14]:
scene.preview()

Renderer(camera=PerspectiveCamera(aspect=1.31, children=(DirectionalLight(intensity=0.25, position=(0.0, 0.0, …